In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine
import altair as alt
import numpy as np
from dotenv import load_dotenv

In [3]:
load_dotenv()

host = os.getenv("MYSQL_HOST", "localhost")
port = os.getenv("MYSQL_PORT", "3306")
user = os.getenv("MYSQL_USER","root")
password = os.getenv("MYSQL_PASSWORD")
db = os.getenv("MYSQL_DB", "rxnorm")

engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8mb4"
)


In [4]:
def Searchbar(name):
    query = f"""
    SELECT * from CONCEPT
    WHERE STR LIKE "%%{name}%%" and TTY = "BN";
    """
    search_results = pd.read_sql(query, engine)

    return search_results

x = Searchbar("Tylenol")
list(x["STR"]) 

['Tylenol', 'Tylenol PM', 'Tylenol with Codeine']

In [5]:
def Fetch_Drug_Form(name):
    query = f"""
    SELECT RXCUI from CONCEPT
    WHERE STR = "{name}" and TTY = "BN";
    """
    result = pd.read_sql(query, engine)
    ID = result["RXCUI"][0]
    
    query = f"""
    SELECT r.RXCUI2 as Drug_Form_ID,r.RELA as Relation,c.STR as Drug_Form
    from CONCEPT c
    JOIN RXNREL r
    ON c.RXCUI = r.RXCUI2
    WHERE r.RXCUI1 = "{ID}" and c.TTY = "DP"
    GROUP by Drug_Form_ID,Relation,Drug_Form;
    """
    result = pd.read_sql(query, engine)
    
    lst = []
    for i in result["Drug_Form"]:
        d = i.split("[")
        lst.append(d[-1][:-1])
    
    result["Product_Name"] = lst
    
    products = []
    e = []
    for i,j in enumerate(result["Product_Name"]):
        if j.lower() in products:
            e.append(i)
        else:
            products.append(j.lower())
    result = result.drop(e)
        
    return result
    
x = Fetch_Drug_Form("Tylenol PM")
x[["Drug_Form_ID","Drug_Form","Product_Name"]]

,Drug_Form_ID,Drug_Form,Product_Name
0,1092378,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Tylenol PM Extra Strength
1,1092378,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Tylenol PM
2,1092378,DIPHENHYDRAMINE HYDROCHLORIDE 25 mg / ACETAMIN...,"Tylenol PM Extra Strength, CVP HEALTH"
3,1092378,DIPHENHYDRAMINE HYDROCHLORIDE 25 mg / ACETAMIN...,Lil Drug Store Tylenol PM Extra Strength


In [6]:
def Show_Ingredients(ID):
    query = f"""
    SELECT r.RXCUI2 as Ingredient_ID,r.RELA as Relation,c.STR as Ingredient
    from CONCEPT c
    JOIN RXNREL r
    ON c.RXCUI = r.RXCUI2
    WHERE r.RXCUI1 = "{ID}" and c.TTY = "SCDC"
    GROUP by Ingredient_ID,Relation,Ingredient;
    """
    result1 = pd.read_sql(query, engine)
    return result1

df = Show_Ingredients(1092378)
ingredients = list(df["Ingredient_ID"])
df

,Ingredient_ID,Relation,Ingredient
0,315266,constitutes,acetaminophen 500 MG
1,901813,constitutes,diphenhydramine hydrochloride 25 MG


In [7]:
def Dose_Form(ID):
    query = f"""
    SELECT r.RXCUI2 as Ingredient_ID,r.RELA as Relation,c.STR as Ingredient
    from CONCEPT c
    JOIN RXNREL r
    ON c.RXCUI = r.RXCUI2
    WHERE r.RXCUI1 = "{ID}" and c.TTY = "DF"
    GROUP by Ingredient_ID,Relation,Ingredient;
    """
    result = pd.read_sql(query, engine)
    DF = result["Ingredient"][0]
    return DF
Dose_Form(209387)

'Oral Tablet'

In [8]:
def get_generic(ID):
    query = f"""
    SELECT r.RXCUI2 as Ingredient_ID,c.STR as Ingredient
    from CONCEPT c
    JOIN RXNREL r
    ON c.RXCUI = r.RXCUI2
    WHERE r.RXCUI1 = "{ID}" and c.TTY = "SCD"
    GROUP by Ingredient_ID,Ingredient;
    """
    res = pd.read_sql(query, engine)
    return res
    
get_generic(209387)

,Ingredient_ID,Ingredient
0,313782,acetaminophen 325 MG Oral Tablet


In [12]:
def Exact_Drugs(Ing_lst,ID):
    s = ""
    for i,j in enumerate(Ing_lst):
        if i == (len(Ing_lst) - 1):
            s+="r1.RXCUI1 = "+j
        else:
            s+="r1.RXCUI1 = "+j+" or "
            
    query = f"""
    WITH base AS (
        SELECT r2.RXCUI as ID, r2.STR as DP, r1.RXCUI1 as Ingredient_ID
        FROM RXNREL r1
        JOIN RXNCONSO r2
        ON r1.RXCUI2 = r2.RXCUI
        WHERE ({s}) and r2.TTY = "DP"
    ),
    keys_all AS (
        SELECT ID
        FROM base
        GROUP by ID
        HAVING COUNT(DISTINCT Ingredient_ID) = {len(Ing_lst)}
    )
    SELECT b.ID,b.DP
    FROM base b
    JOIN keys_all k
    ON b.ID = k.ID
    WHERE b.Id != {ID}
    GROUP BY b.ID, b.DP
    """
    
    res = pd.read_sql(query, engine)
    
    lst = []
    drp = []
    for j,i in enumerate(res["DP"]): 
        if "[" in i:
            d = i.split("[")
            lst.append(d[-1][:-1])
        else:
            lst.append("Generic")
            drp.append(j)

    res["Product_Name"] = lst
    
    Product = []
    for j,i in enumerate(res["Product_Name"]):
        if i.lower() in Product:
            drp.append(j)
        else:
            Product.append(i.lower())
    res = res.drop(drp)
    res = res.reset_index(drop=True)
    return res
    
df = Exact_Drugs(ingredients,1092378)
df

,ID,DP,Product_Name
0,1094718,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,PANADOL PM
1,1117245,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,PAIN RELIEVER PM Extra Strength
2,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,EXTRA STRENGTH PAIN RELIEF PM
3,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Pain Relief PM Extra Strength
4,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,CareOne Acetaminophen PM
5,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Acetaminophen PM Extra Strength
6,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Pain Relief Acetaminophen PM Extra Strength
7,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,good sense pain relief pm extra strength
8,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Topcare pain relief PM extra strength non habi...
9,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,dg health pain relief pm extra strength


In [10]:
def Union_Drugs(Ing_lst,ID):
    s = ""
    for i,j in enumerate(Ing_lst):
        if i == (len(Ing_lst) - 1):
            s+="r1.RXCUI1 = "+j
        else:
            s+="r1.RXCUI1 = "+j+" or "
            
    query = f"""
    WITH base AS (
        SELECT r2.RXCUI as ID, r2.STR as DP, r1.RXCUI1 as Ingredient_ID
        FROM RXNREL r1
        JOIN RXNCONSO r2
        ON r1.RXCUI2 = r2.RXCUI
        WHERE ({s}) and r2.TTY = "DP"
    ),
    keys_all AS (
        SELECT ID
        FROM base
        GROUP by ID
        HAVING COUNT(DISTINCT Ingredient_ID) < {len(Ing_lst)}
    )
    SELECT b.ID,b.DP
    FROM base b
    JOIN keys_all k
    ON b.ID = k.ID
    WHERE b.Id != {ID}
    GROUP BY b.ID, b.DP
    """
    
    res = pd.read_sql(query, engine)
    
    lst = []
    drp = []
    for j,i in enumerate(res["DP"]): 
        if "[" in i:
            d = i.split("[")
            lst.append(d[-1][:-1])
        else:
            lst.append("Generic")
            drp.append(j)

    res["Product_Name"] = lst
    
    Product = []
    for j,i in enumerate(res["Product_Name"]):
        if i.lower() in Product:
            drp.append(j)
        else:
            Product.append(i.lower())
    res = res.drop(drp)
    res = res.reset_index(drop=True)
    return res
    
df = Union_Drugs(ingredients,1092378)
df

,ID,DP,Product_Name
0,1358835,ACETAMINOPHEN 500 mg ORAL TABLET [CETAFEN EXTRA],CETAFEN EXTRA
1,404172,ACETAMINOPHEN 500 mg / CAFFEINE 65 mg ORAL TAB...,Excedrin Tension Headache
2,1052641,ACETAMINOPHEN 500 mg / PAMABROM 25 mg / PYRILA...,Premsyn pms Premenstrual Pain Relief
3,1092416,DIPHENHYDRAMINE HYDROCHLORIDE 25 mg ORAL TABLE...,Nytol
4,1089968,ACETAMINOPHEN 500 mg / CHLORPHENIRAMINE MALEAT...,Extra Strength TYLENOL Cold plus Flu Multi-Act...
...,...,...,...
365,1550957,DIPHENHYDRAMINE HYDROCHLORIDE 25 mg / NAPROXEN...,equaline naproxen sodium pm
366,1550957,DIPHENHYDRAMINE HYDROCHLORIDE 25 mg / NAPROXEN...,topcare all night pain relief pm
367,1550957,DIPHENHYDRAMINE HYDROCHLORIDE 25 mg / NAPROXEN...,good sense naproxen sodium
368,1550957,NAPROXEN SODIUM 220 mg / DIPHENHYDRAMINE HYDRO...,Naproxen Sodium and Diphenhydramine HCl


In [15]:
def Fetch_Ingredients(ID):
    query = f"""
    SELECT c.STR as Ingredient
    from CONCEPT c
    JOIN RXNREL r
    ON c.RXCUI = r.RXCUI2
    WHERE r.RXCUI1 = "{ID}" and c.TTY = "SCDC"
    GROUP by Ingredient;
    """
    result1 = pd.read_sql(query, engine)
    return list(result1["Ingredient"])

Fetch_Ingredients(1092378)

['acetaminophen 500 MG', 'diphenhydramine hydrochloride 25 MG']

In [16]:
def Fetch_Ingredient_Name(ID):
    query = f"""
    SELECT STR as Ingredient
    from RXNCONSO
    WHERE RXCUI = "{ID}" and TTY = "SCDC"
    GROUP by Ingredient;
    """
    result = pd.read_sql(query, engine)
    return result["Ingredient"][0]

Fetch_Ingredient_Name(315266)

'acetaminophen 500 MG'

In [19]:
df = df.head(10)

In [20]:
def Fetch_Heatmap(df):
    Ids = {}
    for i in df["ID"]:
        if i in Ids:
            continue
        else:
            Ids[i] = Fetch_Ingredients(i)
    
    all_ings = []
    for i in Ids:
        for j in Ids[i]:
            if j not in all_ings:
                all_ings.append(j)
                
    print(all_ings)
    print(len(all_ings))
    
    for i,j in enumerate(all_ings):
        lst = []
        for k in df["ID"]:
            if j in Fetch_Ingredients(k):
                lst.append(1)
            else:
                lst.append(0)
        print(lst)
        df[all_ings]=lst
    return df

Fetch_Heatmap(df)
        

['acetaminophen 500 MG', 'diphenhydramine hydrochloride 25 MG']
2
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


ValueError: Columns must be same length as key

,ID,DP,Product_Name
0,1094718,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,PANADOL PM
2,1117245,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,PAIN RELIEVER PM Extra Strength
3,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,EXTRA STRENGTH PAIN RELIEF PM
4,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Pain Relief PM Extra Strength
6,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,CareOne Acetaminophen PM
7,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Acetaminophen PM Extra Strength
8,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Pain Relief Acetaminophen PM Extra Strength
14,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,good sense pain relief pm extra strength
15,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Topcare pain relief PM extra strength non habi...
16,1092189,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,dg health pain relief pm extra strength
